In [2]:
import numpy as np
import os
from numpy.linalg import inv

In [5]:
input_dir = '/ssd2/tingting/HFMLNewFiles-old-parsed/trigger/1'
# /ssd2/tingting/HFMLNewFiles-old-parsed/nontrigger/0
n_files = 10
filenames = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir)
                                if f.startswith('event') and not f.endswith('_ID.npz')])[500000:500000+n_files]

In [3]:
def matmul_3D(A, B):
    return np.einsum('lij,ljk->lik', A, B)

def get_approximate_radius(tracks_info, is_complete):
    complete_tracks = tracks_info[is_complete]
    # complete_track_momentums = track_momentum[is_complete]
    A = np.ones((complete_tracks.shape[0], 5, 3))
    A[:, :, 0] = complete_tracks[:, [0, 3, 6, 9, 12]]
    A[:, :, 1] = complete_tracks[:, [1, 4, 7, 10, 13]]
    y = - complete_tracks[:, [0, 3, 6, 9, 12]]**2 - complete_tracks[:, [1, 4, 7, 10, 13]]**2
    y = y.reshape((y.shape[0], y.shape[1], 1))
    # c = np.einsum('lij,ljk->lik', inv(A), y)
    AT = np.transpose(A, axes=(0, 2, 1))
    # print(A.shape, AT.shape, y.shape)
    # c = inv(matmul_3D(A, AT))
    c = matmul_3D(matmul_3D(inv(matmul_3D(AT, A)), AT), y)
    # print(A.shape, AT.shape, y.shape, c.shape)
    r = np.sqrt(c[:, 0]**2 + c[:, 1]**2 - 4*c[:, 2])/200
    return r

def get_approximate_radii(tracks_info, n_hits, good_hits):
    x_indices = [3*j for j in range(5)]
    y_indices = [3*j+1 for j in range(5)]
    r = np.zeros((tracks_info.shape[0], 1))
    for n_hit in range(3, 5 + 1):
        complete_tracks = tracks_info[n_hits == n_hit]
        hit_indices = good_hits[n_hits == n_hit]
        if complete_tracks.shape[0] == 0:
            continue

        A = np.ones((complete_tracks.shape[0], n_hit, 3))
        x_values = complete_tracks[:, x_indices]
        x_values = x_values[hit_indices].reshape(complete_tracks.shape[0], n_hit)

        y_values = complete_tracks[:, y_indices]
        y_values = y_values[hit_indices].reshape(complete_tracks.shape[0], n_hit)
        A[:, :, 0] = x_values
        A[:, :, 1] = y_values

        y = - x_values**2 - y_values**2
        y = y.reshape((y.shape[0], y.shape[1], 1))
        AT = np.transpose(A, axes=(0, 2, 1))
        c = matmul_3D(matmul_3D(inv(matmul_3D(AT, A)), AT), y)
        r[n_hits == n_hit] == 1
        r[n_hits == n_hit] = np.sqrt(c[:, 0]**2 + c[:, 1]**2 - 4*c[:, 2])/200
    #test = get_approximate_radius(tracks_info, n_hits == 5)
    #assert np.allclose(test, r[n_hits == 5])

    return r

def get_predicted_pz(first_hit, last_hit, radius):
    dz = (last_hit[:, -1] - first_hit[:, -1])/100
    chord2 = ((last_hit[:, 0] - first_hit[:, 0]) ** 2 + (last_hit[:, 1] - first_hit[:, 1]) ** 2) / 10000
    dtheta = np.arccos((2*radius**2 - chord2) / (2*radius**2 + 1e-10))
    return np.nan_to_num(dz / dtheta)

def load_graph(filename, load_compelete_graph=False):
    with np.load(filename) as f:
        complete_flags = f['complete_flags'] 
        if load_compelete_graph and len(complete_flags)!=0:
            track_vector = f['track_vector'][complete_flags]
            origin_vertices = f['origin_vertices'][complete_flags]
            momentums = f['momentums'][complete_flags].reshape(-1, 3)
            pids = f['pids'][complete_flags]
            ptypes = f ['ptypes'][complete_flags]
            energy = f['energy'][complete_flags]
            trigger_track_flag = f['trigger_track_flag'][complete_flags]
        else:
            track_vector = f['track_vector']
            origin_vertices = f['origin_vertices']
            momentums = f['momentums'].reshape(-1, 3)
            pids = f['pids']
            ptypes = f ['ptypes']
            energy = f['energy']
            trigger_track_flag = f['trigger_track_flags']
        trigger = f['trigger']
        ip = f['ip']
        valid_trigger_flag = f['valid_trigger_flag']
        n_track = track_vector.shape[0]

    return track_vector, complete_flags, origin_vertices, momentums, pids, ptypes, energy, trigger, ip, trigger_track_flag, valid_trigger_flag

def get_length(start, end):
    return np.sqrt(np.sum((start - end)**2, axis=1))

In [6]:
for file_index in range(len(filenames)):
    track_vector, complete_flags, origin_vertices, momentums, pids, ptypes, energy, trigger, ip, trigger_track_flag, valid_trigger_flag = load_graph(filenames[file_index])
    
    # calculate predicted radius
    hits = track_vector[:, :15].reshape(track_vector.shape[0], 5, 3)
    good_hits = np.all(hits != 0, axis=-1)
    n_hits = np.sum(good_hits, axis=-1)
    r = get_approximate_radii(track_vector, n_hits, good_hits).reshape(-1, 1)

    # calculate predicted pz
    first_hit = [0] * good_hits.shape[0]
    last_hit = [0] * good_hits.shape[0]
    for i in range(good_hits.shape[0]):
        good_hits_index = np.nonzero(good_hits[i])[0]
        if len(good_hits_index) > 0:
            first_hit[i], last_hit[i] = good_hits_index[0], good_hits_index[-1]
    pred_pz = get_predicted_pz(hits[np.arange(good_hits.shape[0]), first_hit], hits[np.arange(good_hits.shape[0]), last_hit], r.reshape(-1)).reshape(-1, 1)

    # ground truth pt
    pt = np.sqrt(momentums[:, 0] ** 2 + momentums[:, 1] ** 2).reshape(-1, 1)

    # ground truth pz
    pz = (momentums[:, 2]).reshape(-1, 1)

    # print(r, pred_pz, pt, pz)
    print(pids)

[ 5  6 54 69 70 71 73 74 78 80 82 83 90 91 92]
[ 2  9 10 13 44 49 51 56 57 59 62 63 66 67 68 72]
[-123    7    8   17   18   19   30   31   36   37   38   44   45   46]
[-1523 -1452 -1010  -946     8    10    12    23    24    58    59    65
    73    75    76    77    79    80    81    82    83    84    87    91
    94    95    96]
[-363    2    3    5    6    7   48   53   57   58   59   64   70   71
   76   79]
[ 2  7  8 42 43 49 50 58 61 62 63 73 74 75 83 84 85 86 91 94 95]
[ 3  4 11 20]
[-6966 -6781    10    13    14    27    28    42    43    45    46    47
    51    52]
[-69   5   7   8  40  49  51  52  54  55  57  59  63  64]
[11 12 23 31 38 39 40 41]


/tmp/ipykernel_3001785/3656505793.py:54: RuntimeWarning: invalid value encountered in arccos
  dtheta = np.arccos((2*radius**2 - chord2) / (2*radius**2 + 1e-10))
